# Preprocessing of the Scaped Review Data

## Steps

### 1.  Importing Libraries along with our Data
### 2. Expanding Contractions
### 3. Tokenization
### 4. Removing Punctuations
### 5. Converting all Characters to Lowercase
### 6. Removing Stopwords
### 7. Lemmatization

In [1]:
#import  liabraries
import pandas as pd
import string
#import contractions
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import re
import pandas_profiling

In [2]:
with open('Indeed_scrape.csv') as f:
    df = pd.read_csv(f)
f.close()

In [3]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [4]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [5]:
pd.set_option('max_colwidth',50)
df.head()

,Unnamed: 0,rating,rating_title,rating_description,rating_pros,rating_cons
0,0,5.0,Technical Support Mississauga Ontario,Customer service/call centre mainly tech suppo...,NaN,NaN
1,1,5.0,Technical Support Mississauga Ontario,Customer service/call centre mainly tech suppo...,NaN,NaN
2,2,4.0,great place to work,"the management is great, the culture is awesom...","managment, people, culture",money
3,3,2.0,Lots of politics,The work culture has changed so that there is ...,Free lunches,Unfulfilling work
4,4,3.0,Productive company,A typical day at work in the company is fast p...,NaN,NaN


In [6]:
df.dtypes

Unnamed: 0              int64
rating                float64
rating_title           object
rating_description     object
rating_pros            object
rating_cons            object
dtype: object

In [7]:
df.describe()

,Unnamed: 0,rating
count,4142.000000,4142.000000
mean,2070.500000,4.663930
std,1195.836736,0.645268
min,0.000000,2.000000
25%,1035.250000,5.000000
50%,2070.500000,5.000000
75%,3105.750000,5.000000
max,4141.000000,5.000000


In [8]:
df = df[['rating', 'rating_description']]

In [9]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9/\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [10]:
 df['rating_description']= df['rating_description'].apply(lambda x: remove_special_characters(x))

In [11]:
df['no_contract_int'] = df['rating_description'].apply(lambda x: [expand_contractions(word) for word in x.split()])

In [12]:
#df['no_contract'] = df['no_contract_int'].apply(lambda x : for word in x.split('/'))

In [13]:
df['description_back'] = [' '.join(map(str,l)) for l in df['no_contract_int']]

In [14]:
df['tokenized']= df['description_back'].apply(word_tokenize)

In [15]:
#df['lower'] = df['tokenized'].apply(lambda x:[word.split('/') for word in x])
df['lower'] = df['tokenized'].apply(lambda x:[word.lower() for word in x])

In [16]:
#df['split'] = df['lower'].apply(lambda x: word.split('/') for word in x)

In [17]:
punc = string.punctuation

In [18]:
df['no_punc'] = df['lower'].apply(lambda x:[word for word in x if word not in punc])

In [19]:
stop_words = set(stopwords.words('english'))

In [20]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [21]:
df['stopwords_removed']=df['no_punc'].apply(lambda x:[word for word in x if word not in stop_words])

In [22]:
df['pos_tags'] =  df['stopwords_removed'].apply(nltk.tag.pos_tag)

In [23]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tarun.bagga\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tarun.bagga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [26]:
df['wordnet_pos'] = df['pos_tags'].apply(lambda x:[(word,get_wordnet_pos(pos_tag)) for (word,pos_tag) in x])

In [27]:
wnl = WordNetLemmatizer()
df['lemmatized']=df['wordnet_pos'].apply(lambda x:[wnl.lemmatize(word,tag) for word,tag in x])

In [28]:
df.to_pickle('df_clean.pkl')